## 기본 파이썬

In [1]:
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

In [2]:
input_file = './data/sales_2013.xlsx'
output_file = './output_files/03-2 output.xls'

In [3]:
output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('selected_columns_all_worksheets')

In [4]:
my_columns = ['Customer Name', 'Sale Amount']

In [5]:
first_worksheet = True
with open_workbook(input_file) as workbook:
    data = [my_columns]
    index_of_cols_to_keep = []
    for worksheet in workbook.sheets():
        if first_worksheet:
            header = worksheet.row_values(0)
            for column_index in range(len(header)):
                if header[column_index] in my_columns:
                    index_of_cols_to_keep.append(column_index)
            first_worksheet = False
        for row_index in range(1, worksheet.nrows):
            row_list = []
            for column_index in index_of_cols_to_keep:
                cell_value = worksheet.cell_value(row_index, column_index)
                cell_type = worksheet.cell_type(row_index, column_index)
                if cell_type == 3:
                    date_cell = xldate_as_tuple(cell_value, workbook.datemode)
                    date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                    row_list.append(date_cell)
                else:
                    row_list.append(cell_value)    
            data.append(row_list)
    for list_index, output_list in enumerate(data):
        for element_index, element in enumerate(output_list):
            output_worksheet.write(list_index, element_index, element)
output_workbook.save(output_file)

### 팬더스

In [6]:
import pandas as pd

output_file = './output_files/03-2 pandas_output.xls'

In [7]:
data_frame = pd.read_excel(input_file, sheet_name=None, index_col=None)

column_output = []

for worksheet_name, data in data_frame.items():
    column_output.append(data.loc[:, ['Customer Name', 'Sale Amount']])
selected_columns = pd.concat(column_output, axis=0, ignore_index=True)

In [8]:
writer = pd.ExcelWriter(output_file)

selected_columns.to_excel(writer, sheet_name='selected_columns_all_worksheets', index=False)

writer.save()